[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/weaviate/recipes/weaviate-features/model-providers/aws/rag_titan-text-express-v1_bedrock.ipynb)

# RAG with AWS Nova Lite on Bedrock and Weaviate

This notebook will show you how to use the dynamic RAG API to define the model provider at query time. This enables you to change between different models and model settings at query time. 

## Dependencies

In [ ]:
import weaviate
from weaviate.classes.config import Property, DataType
import weaviate.classes.config as wc
from weaviate.classes.generate import GenerativeConfig

import os
import requests
import json

In [ ]:
!pip install --quiet weaviate-client==4.16.9

## Step 1: Connect to Weaviate

**Connect to your running Weaviate cluster (choose one option)**
1. You can create a 14-day free sandbox on [WCD](https://console.weaviate.cloud/)
2. [Embedded Weaviate](https://weaviate.io/developers/weaviate/installation/embedded)
3. [Local deployment](https://weaviate.io/developers/weaviate/installation/docker-compose#starter-docker-compose-file)
4. [Other options](https://weaviate.io/developers/weaviate/installation)

Option 1: Weaviate Cloud

In [ ]:
WCD_URL = os.environ.get("WEAVIATE_URL")  # Replace with your Weaviate cluster URL
WCD_AUTH_KEY = os.environ.get("WEAVIATE_AUTH")  # Replace with your cluster auth key
AWS_ACCESS_KEY = os.environ.get("AWS_ACCESS_KEY")  # Replace with your access key 
AWS_SECRET_KEY = os.environ.get("AWS_SECRET_KEY")  # Replace with your secret key 

# Weaviate Cloud Deployment
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WCD_URL,
    auth_credentials=weaviate.auth.AuthApiKey(WCD_AUTH_KEY),
    headers = {
    "X-AWS-Access-Key": AWS_ACCESS_KEY,
    "X-AWS-Secret-Key": AWS_SECRET_KEY,
    }
)

print(client.is_ready())

True


Option 2: Weaviate Embedded

In [ ]:
# client = weaviate.connect_to_embedded()

# print(client.is_ready())

Option 3: Weaviate local host

In [ ]:
# client = weaviate.connect_to_local()

# client.close()

## Create a collection
> Collection stores your data and vector embeddings.

Rather than defining the generative model provider at collection creation, we will define the provider at query time.

In [ ]:
# Note: in practice, you shouldn"t rerun this cell, as it deletes your data
# in "JeopardyQuestion", and then you need to re-import it again.

# Delete the collection if it already exists
if (client.collections.exists("JeopardyQuestion")):
    client.collections.delete("JeopardyQuestion")

client.collections.create(
    name="JeopardyQuestion",

    vectorizer_config=wc.Configure.Vectorizer.text2vec_aws(
        service="bedrock",   #this is crucial
        model="amazon.titan-embed-text-v2:0", # select the model, make sure it is enabled for your account
        region="us-east-1" # select your region
    ),

    properties=[ # defining properties (data schema) is optional
        wc.Property(name="Question", data_type=wc.DataType.TEXT), 
        wc.Property(name="Answer", data_type=wc.DataType.TEXT),
        wc.Property(name="Category", data_type=wc.DataType.TEXT, skip_vectorization=True), 
    ]
)

print("Successfully created collection: JeopardyQuestion.")

## Import the Data

In [ ]:
url = "https://raw.githubusercontent.com/weaviate/weaviate-examples/main/jeopardy_small_dataset/jeopardy_tiny.json"
resp = requests.get(url)
data = json.loads(resp.text)

# Get a collection object for "JeopardyQuestion"
jeopardy = client.collections.get("JeopardyQuestion")

# Insert data objects
response = jeopardy.data.insert_many(data)

# Note, the `data` array contains 10 objects, which is great to call insert_many with.
# However, if you have a milion objects to insert, then you should spit them into smaller batches (i.e. 100-1000 per insert)

if (response.has_errors):
    print(response.errors)
else:
    print("Insert complete.")

## Generative Search Queries

We will define the generative model at query time. Notice how we're using two different models for each query.

### Single Result

Single Result makes a generation for each individual search result. 

In the below example, I want to create a Facebook ad from the Jeopardy question about Elephants. 

In [ ]:
gen_provider = GenerativeConfig.aws(service="bedrock", model="amazon.nova-lite-v1:0", region= "us-east-1") # change these variables to match the model you deployed on Bedrock


generatePrompt = "Turn the following Jeogrady question into a Facebook Ad: {question}"

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.generate.near_text(
    query="Elephants",
    limit=2,
    single_prompt=generatePrompt,
    generative_provider=gen_provider
)

for item in response.objects:
    print(json.dumps(item.properties, indent=1))
    print("-----vvvvvv-----")
    print(item.generated)
    print("-----^^^^^^-----")

{
 "category": "ANIMALS",
 "question": "It's the only living mammal in the order Proboseidea",
 "answer": "Elephant"
}
-----vvvvvv-----
**🌟 Discover the Marvel of Nature! 🌟**

🐘 Did you know there's only one living mammal in the order Proboscidea? 🐘

Introducing the majestic **Elephant**! 🌿🌍

🌟 From the ancient forests to modern savannas, elephants have roamed the Earth for millions of years. They are the largest land animals on our planet, known for their incredible intelligence, deep social bonds, and unique trunk that serves as a nose, a hand, and a tool for survival.

🌟 Join us in celebrating these gentle giants and learn more about their fascinating lives and the importance of protecting their habitats. 

🌱 **Support Conservation Efforts Today!** 🌱

👉 Click the link to learn more and contribute to the preservation of these incredible creatures!

🔗 [Learn More & Donate] 

#Elephants #WildlifeConservation #NatureLovers #Proboscidea #ProtectOurPlanet
-----^^^^^^-----
{
 "category": "

### Grouped Result

Grouped Result generates a single response from all the search results. 

The below example is creating a Facebook ad from the 2 retrieved Jeoprady questions about animals. 

In [ ]:
gen_provider = GenerativeConfig.aws(service="bedrock", model="amazon.nova-pro-v1:0", region= "us-east-1") # change these variables to match the model you deployed on Bedrock

generateTask = "Explain why these Jeopardy questions are under the Animals category."

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.generate.near_text(
    query="Animals",
    limit=3,
    grouped_task=generateTask,
    generative_provider=gen_provider
)

print(response.generated)

The questions provided are categorized under "ANIMALS" for several reasons:

1. **Content Focus**: Each question directly pertains to a specific animal or a characteristic of an animal. The questions ask about elephants, the anatomical feature of the gavial (a type of crocodile), and antelopes, all of which are animals.

2. **Biological Classification**: The first question references the biological classification of the elephant, which is a mammal in the order Proboseidea. This type of question requires knowledge of zoology and animal taxonomy.

3. **Anatomical Features**: The second question discusses a specific anatomical feature of the gavial, which is a type of crocodile. This again requires knowledge about animal anatomy and differences between species.

4. **Species Identification**: The third question is about the eland, a species of antelope, and its relative size compared to other antelopes in Africa. This involves knowledge of animal species and their habitats.

Given these p